# Plot all PACE from eddy of interest

Authors: SEL, LJK 

In [14]:
import earthaccess
import xarray as xr
from xarray.backends.api import open_datatree
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import matplotlib.animation as animation
import os

In [15]:
auth = earthaccess.login(persist=True)

### TO DO: Switch out specialid for correct eddy ID based on AVISO eddy product. Switch out AVISO eddy filename below for correct one

In [16]:
specialid='eddy_ID164996'
aviso_ds = xr.open_dataset('Eddy_trajectory_nrt_3.2exp_cyclonic_20180101_20241111_ID_164996.nc')

In [17]:
dates = [i for i in aviso_ds.time.values] # these are sorted already
print(min(dates))
print(max(dates)) # still existed at the end of the dataset
center_lats = [i for i in aviso_ds.latitude.values]
center_lons = [i for i in aviso_ds.longitude.values]
print(len(center_lats))

2023-08-21T00:00:00.000000000
2024-09-05T00:00:00.000000000
382


In [7]:
tminny=str(min(dates))
tminny=tminny[0:10]
tmaxxy=str(max(dates))
tmaxxy=tmaxxy[0:10]
tmin,tmax = tminny,tmaxxy

In [8]:
center_lats = [i for i in aviso_ds.latitude.values]
center_lons = [i for i in aviso_ds.longitude.values]
print(len(center_lats))

382


In [9]:
#Define box based on eddy
latmin,latmax = min(center_lats)-2,max(center_lats)+2
lonmin,lonmax = min(center_lons)-360-2,max(center_lons)-360+2

In [10]:
# tspan = ("2024-04-01", "2024-09-20")
tspan=(tmin,tmax)
clouds = (0, 100)
bbox = (lonmin, latmin, lonmax, latmax) #edward

In [11]:
results = earthaccess.search_data(
    short_name="PACE_OCI_L2_BGC_NRT",
    temporal=tspan,
    bounding_box=bbox,
    cloud_cover=clouds,
)

In [13]:
paths = earthaccess.open(results) #'streaming' data

In [ ]:
def get_eddy_by_ID_date(ds,eddy_date):
    """
    ds: netCDF AVISO format
    track_id: id of eddy to extract
    eddy_date: date in format 'YYYY-MM-DD'

    Returns contour lons, contour lats, center lon, center lat
    """
    try:
        ind = np.where(ds.time == np.datetime64(eddy_date))[0][0]
        contour_lons = np.array(ds.effective_contour_longitude[ind])
        contour_lats = np.array(ds.effective_contour_latitude[ind])
    except:
        print('No eddy data available with that request ... :(')        
    
    return contour_lons,contour_lats,ds.longitude[ind],ds.latitude[ind]

In [ ]:
for index in range(0,len(paths)):
    datatree = open_datatree(paths[index])
    datatree
    dataset = xr.merge(datatree.to_dict().values())
    dataset
    dataset = dataset.set_coords(("longitude", "latitude"))

    h=str(paths[index])
    datey=h[64:72]
    
    date_requested=datey
    contour_lons,contour_lats,center_lon,center_lat = get_eddy_by_ID_date(aviso_ds,'%s-%s-%s'%(date_requested[0:4],date_requested[4:6],date_requested[6:8]))

    fig = plt.figure()
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.gridlines(draw_labels={"left": "y", "bottom": "x"})
    plot = dataset["chlor_a"].plot(x="longitude", y="latitude", cmap="Spectral_r", vmax=0.5, ax=ax)
    ax.set_ylabel('latitude')
    ax.set_xlabel('longitude')
    ax.set_ylim([latmin,latmax])
    ax.set_xlim([lonmin,lonmax])
    ax.plot(contour_lons-360,contour_lats,zorder=100,c='k',linewidth=2)
    ax.set_title(datey[0:4]+'-'+datey[4:6]+'-'+datey[6:len(datey)])
    if len(str(index))==1:
        plt.savefig(str(specialid)+'CHL'+str(0)+str(0)+str(index).split('.')[0]+'pace.png')
    elif len(str(index))==2:
        plt.savefig(str(specialid)+'CHL'+str(0)+str(index).split('.')[0]+'pace.png')
    else:
        plt.savefig(str(specialid)+'CHL'+str(index).split('.')[0]+'pace.png')
    plt.close()

In [ ]:
# Get a list of all PNG files in the directory
image_dir = '/home/jovyan/GO-SWACE/'
images = [img for img in os.listdir(image_dir) if img.endswith("pace.png")]
images.sort()  # Sort the images if needed
# Create a figure and axis for the animation
fig, ax = plt.subplots()

# Function to update the figure for each frame
def update(frame):
    img_path = os.path.join(image_dir, images[frame])
    img = plt.imread(img_path)
    ax.imshow(img)
    ax.axis('off')  # Hide axes
# Create the animation
ani = animation.FuncAnimation(fig, update, frames=len(images), repeat=True)

# Save the animation as a GIF or display it
# ani.save('animation_edward_all.gif', writer='imagemagick', fps=3)  # Save as GIF #was fps=2
ani.save('animation_PACE_' + str(specialid) + '.gif', writer='imagemagick', fps=4)  # fps is frames per second. Lower numbers = slower speeds

# plt.show()  # Uncomment to display the animation

print("Animation created successfully.")

In [ ]:
# image_dir = '/home/jovyan/tutorials/'
# images = [img for img in os.listdir(image_dir) if img.endswith(".png")]
# images

# for index in range(0,len(images)):
#     filename=images[index]
#     if len(images[index])==12:
#         newfilename=filename[0:6]+str(0)+filename[6:12]
#         os.rename(filename, newfilename)
#     elif len(images[index])==11:
#         newfilename=filename[0:6]+str(0)+str(0)+filename[6:12]
#         os.rename(filename, newfilename)

# Single Frame Chl 

In [ ]:
# # Set default fontsizes for plots
# fontsize = 18

# plt.rc('font', size=fontsize)          # controls default text sizes
# plt.rc('axes', titlesize=fontsize)     # fontsize of the axes title
# plt.rc('axes', labelsize=fontsize)    # fontsize of the x and y labels
# plt.rc('xtick', labelsize=fontsize)    # fontsize of the tick labels
# plt.rc('ytick', labelsize=fontsize)    # fontsize of the tick labels
# plt.rc('legend', fontsize=fontsize)    # legend fontsize
# plt.rc('figure', titlesize=fontsize)  # fontsize of the figure title

In [ ]:
# def find_edward_by_date(date_to_plot):
#     """
#     date_to_plot: format "yyyy-mm-dd"
#     """
#     tspan = (date_to_plot, date_to_plot)
#     bbox = (-73, 35, -70, 37) #edward

#     try:
#         results = earthaccess.search_data(
#             short_name="PACE_OCI_L2_BGC_NRT",
#             temporal=tspan,
#             bounding_box=bbox)
#         paths = earthaccess.open(results) #'streaming' data
#         datatree = open_datatree(paths[0]) # could be multiple paths on the same day, just taking one
#         dataset = xr.merge(datatree.to_dict().values())
#     except:
#         print('No Edward data today :(')
#         dataset = None
    
#     return dataset

In [ ]:
# dataset = find_edward_by_date("2024-04-14")

In [ ]:
# fig.clf()
# fig,ax=plt.subplots(1,1,figsize=(10,7))

# plot=ax.pcolormesh(dataset["longitude"],dataset["latitude"],dataset["chlor_a"],cmap='Spectral_r',vmax=0.5)
# cbar = plt.colorbar(plot,ax=ax)
# cbar.set_label('Chl-a (mg $ \cdot m^{-3}$)', rotation=270, labelpad=20)

# ax.set_ylim([33,38])
# ax.set_xlim([-73,-67])


# ax.set_title(date_to_plot)
# ax.set_ylabel('Latitude (N$^{\circ}$)')
# ax.set_xlabel('Longitude (E$^{\circ}$)')

# #plt.savefig('Edward_chl_%s.png'%(date_to_plot),dpi=300)

# Double Frame Chl 

In [ ]:
# dataset_0414 = find_edward_by_date("2024-04-14")
# dataset_0508 = find_edward_by_date("2024-05-08")

In [ ]:
# #fig.clf()
# fig,ax=plt.subplots(1,2,figsize=(14,5))
# plt.subplots_adjust(wspace=0.1)

# a = 0
# for ds in [dataset_0414,dataset_0508]:

#     #plot=ax[a].pcolormesh(ds["longitude"],ds["latitude"],np.log(ds["chlor_a"]),cmap='ocean_r',vmin=-2,vmax=-0.7)
#     plot=ax[a].pcolormesh(ds["longitude"],ds["latitude"],ds["chlor_a"],cmap='Spectral_r',vmin=0.1,vmax=0.45)
#     a += 1

# #cbar = plt.colorbar(plot,ax=ax)

# cbar = fig.colorbar(plot, ax=ax, orientation='vertical')
# cbar.set_label('Chl-a (mg $ \cdot m^{-3}$)', rotation=270, labelpad=20)

# ax[0].set_title("14 Apr 2024")
# ax[1].set_title("08 May 2024")

# # Single ax params
# ax[0].set_ylabel('Latitude (N$^{\circ}$)')
# ax[1].set_yticklabels([])

# for a in [0,1]:
#     ax[a].set_ylim([33,38])
#     ax[a].set_xlim([-73,-67])
#     ax[a].set_xlabel('Longitude (E$^{\circ}$)')
#     ax[a].set_facecolor('#e2e2e2')

# #plt.savefig('Edward_chl_0414_0508.png',dpi=300,bbox_inches='tight')